In [ ]:
!pip install -U pyarrow

Requirement already up-to-date: pyarrow in /usr/local/lib/python3.6/dist-packages (2.0.0)


In [ ]:
!pip3 uninstall -y lightgbm

Uninstalling lightgbm-3.1.1:
  Successfully uninstalled lightgbm-3.1.1


In [ ]:
!apt-get install -y -qq libboost-all-dev

In [ ]:
!pip3 install lightgbm --install-option=--gpu --install-option="--opencl-include-dir=/usr/local/cuda/include/" --install-option="--opencl-library=/usr/local/cuda/lib64/libOpenCL.so" 

/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
  Using cached https://files.pythonhosted.org/packages/cf/65/368931f1f234149e14fc6075dbeb1ad8a8aebb105aa11fe8631c72c1bdcf/lightgbm-3.1.1.tar.gz
Skipping wheel build for lightgbm, due to binaries being disabled for it.
    Running setup.py install for lightgbm ... done


In [ ]:
import os
import sys
import time
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import gc
import random
from pickle import dump, load

pd.options.mode.chained_assignment = None

In [ ]:
import lightgbm as lgb

In [ ]:
def read_csv(file_name = "train.csv", dtype = None, skiprows = None, nrows = None, usecols = None):
    data = pd.read_csv(file_name, dtype=dtype, skiprows = skiprows, nrows = nrows, low_memory = True, header = 0, usecols = usecols)
    return data

def read_feather(file_name = "train.feather"):
    data = pd.read_feather(file_name)
    return data

In [ ]:
tdf = read_feather("/content/drive/MyDrive/data/new_mu_sigma_tdf_with_mean_trueskill_hung.feather")

In [ ]:
tdf

,ts_delta,row_id,user_id,content_id,task_container_id,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,part,t1,num_tag,u_attempt_c,bundle_id,total_explained,10_recent_correctness,10_recent_mean_gap,mean_elapsed,mean_gap,this_bundle_elapsed_time_mean,u_chance,u_attempts,c_chance,c_attempts,u_part_chance,u_part_attempts,u_skill_chance,u_skill_attempts,t_chance,t_attempts,prev_cor,prev_t1,trueskill_probability,mu,sigma
0,1689,27908883,595554894,3146,1907,0,43,1,4,142,3,0,1983,2770,0.700195,31035.400391,41,9153,23,0.681998,2783,0.731258,1574,0.724409,381,0.705431,1565,0.688073,33292,1,142,0.935567,0.530848,0.021230
1,1689,27908884,595554894,3144,1907,1,43,1,4,142,3,0,1983,2771,0.600098,15904.400391,41,9153,23,0.681753,2784,0.548920,1574,0.722513,382,0.704981,1566,0.688073,33293,0,142,0.679786,0.525934,0.021166
2,33,47773655,1010145690,4416,76,1,23,1,5,97,1,0,2985,94,0.799805,189.899994,36,111703,19,0.725806,124,0.752521,33619,0.694444,36,0.681818,44,0.789180,45116,1,58,0.933935,0.520507,0.027962
3,41,65764011,1398162955,276,426,1,16,1,2,142,4,2,277,458,0.399902,94.699997,26,1230,18,0.605749,487,0.719678,3603,0.666667,84,0.704403,159,0.688073,33294,1,173,0.668909,0.355612,0.020069
4,23,17025158,364978597,8844,801,1,16,1,5,40,1,0,6321,1078,0.500000,27.200001,42,26729,23,0.747292,1108,0.858166,2235,0.671458,487,0.688126,699,0.576255,44350,1,12,0.961658,0.455220,0.021047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24999995,94,81628033,1737284815,2267,6126,1,15,1,3,157,3,0,1690,9249,0.899902,114.599998,35,845,25,0.683765,9338,0.856007,4653,0.763351,955,0.771445,3579,0.678152,3992,1,142,0.930229,0.415507,0.021346
24999996,18,93531438,1985357445,5592,90,1,9,0,5,43,1,0,4161,0,0.399902,16.600000,17,1797,21,0.595506,89,0.276326,38773,0.620690,29,0.620690,29,0.616443,1164,0,49,0.045668,0.416119,0.027924
24999997,27,51056638,1081757871,3641,443,0,13,1,5,20,1,0,2210,327,0.899902,3750.699951,20,41342,21,0.674847,489,0.604787,29369,0.582781,151,0.572327,159,0.677565,51538,1,4,0.603213,0.441719,0.020535
24999998,23,66241721,1407986221,3592,943,1,39,1,5,67,1,0,2161,876,0.700195,43.599998,40,12415,24,0.624008,1008,0.777648,859,0.651753,827,0.640798,902,0.609270,53159,1,48,0.787565,0.375942,0.020072


In [ ]:
vdf = read_feather("/content/drive/MyDrive/data/new_mu_sigma_vdf_with_mean_trueskill_hung.feather")

In [ ]:
TARGET = 'answered_correctly'
drop_cols = ['row_id', 'user_id', 'content_id', 'bundle_id', 'answered_correctly']
cat_cols = ['prior_question_had_explanation', 'part', 't1', 'num_tag', 'prev_cor', 'prev_t1']

In [ ]:
y_train = tdf[TARGET]
y_val = vdf[TARGET]
tdf.drop(drop_cols, axis=1, inplace=True)
vdf.drop(drop_cols, axis=1, inplace=True)
gc.collect()

0

In [ ]:
tdf.fillna(-1, inplace = True)
vdf.fillna(-1, inplace = True)

In [ ]:
feat_cols = list(tdf.columns)

In [ ]:
tdf

,ts_delta,task_container_id,prior_question_elapsed_time,prior_question_had_explanation,part,t1,num_tag,u_attempt_c,total_explained,10_recent_correctness,10_recent_mean_gap,mean_elapsed,mean_gap,this_bundle_elapsed_time_mean,u_chance,u_attempts,c_chance,c_attempts,u_part_chance,u_part_attempts,u_skill_chance,u_skill_attempts,t_chance,t_attempts,prev_cor,prev_t1,trueskill_probability,mu,sigma
0,1689,1907,43,1,4,142,3,0,2770,0.700195,31035.400391,41,9153,23,0.681998,2783,0.731258,1574,0.724409,381,0.705431,1565,0.688073,33292,1,142,0.935567,0.530848,0.021230
1,1689,1907,43,1,4,142,3,0,2771,0.600098,15904.400391,41,9153,23,0.681753,2784,0.548920,1574,0.722513,382,0.704981,1566,0.688073,33293,0,142,0.679786,0.525934,0.021166
2,33,76,23,1,5,97,1,0,94,0.799805,189.899994,36,111703,19,0.725806,124,0.752521,33619,0.694444,36,0.681818,44,0.789180,45116,1,58,0.933935,0.520507,0.027962
3,41,426,16,1,2,142,4,2,458,0.399902,94.699997,26,1230,18,0.605749,487,0.719678,3603,0.666667,84,0.704403,159,0.688073,33294,1,173,0.668909,0.355612,0.020069
4,23,801,16,1,5,40,1,0,1078,0.500000,27.200001,42,26729,23,0.747292,1108,0.858166,2235,0.671458,487,0.688126,699,0.576255,44350,1,12,0.961658,0.455220,0.021047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24999995,94,6126,15,1,3,157,3,0,9249,0.899902,114.599998,35,845,25,0.683765,9338,0.856007,4653,0.763351,955,0.771445,3579,0.678152,3992,1,142,0.930229,0.415507,0.021346
24999996,18,90,9,0,5,43,1,0,0,0.399902,16.600000,17,1797,21,0.595506,89,0.276326,38773,0.620690,29,0.620690,29,0.616443,1164,0,49,0.045668,0.416119,0.027924
24999997,27,443,13,1,5,20,1,0,327,0.899902,3750.699951,20,41342,21,0.674847,489,0.604787,29369,0.582781,151,0.572327,159,0.677565,51538,1,4,0.603213,0.441719,0.020535
24999998,23,943,39,1,5,67,1,0,876,0.700195,43.599998,40,12415,24,0.624008,1008,0.777648,859,0.651753,827,0.640798,902,0.609270,53159,1,48,0.787565,0.375942,0.020072


In [ ]:
vdf

,ts_delta,task_container_id,prior_question_elapsed_time,prior_question_had_explanation,part,t1,num_tag,u_attempt_c,total_explained,10_recent_correctness,10_recent_mean_gap,mean_elapsed,mean_gap,this_bundle_elapsed_time_mean,u_chance,u_attempts,c_chance,c_attempts,u_part_chance,u_part_attempts,u_skill_chance,u_skill_attempts,t_chance,t_attempts,prev_cor,prev_t1,trueskill_probability,mu,sigma
0,22,824,11,1,2,149,4,0,876,0.899902,20.799999,22,961,18,0.695426,962,0.860809,6157,0.838906,329,0.776139,746,0.731546,59637,1,142,0.887880,0.355988,0.020836
1,274,92,17,1,2,173,5,0,74,0.600098,149.000000,49,4683,16,0.596154,104,0.859472,41902,0.500000,8,0.475000,40,0.763955,30718,0,153,0.903571,0.376694,0.027779
2,55,1019,22,1,2,142,5,0,1897,0.700195,47.799999,50,29398,18,0.699064,1924,0.613618,35408,0.621951,82,0.690789,912,0.689115,3627,0,130,0.730812,0.498266,0.021121
3,87,190,26,1,5,54,1,0,265,0.700195,120.199997,58,137115,23,0.610714,280,0.838405,3286,0.460526,76,0.563380,142,0.628441,50921,1,57,0.935992,0.425220,0.021056
4,56,574,17,1,5,40,1,0,525,0.600098,4998.100098,19,11757,20,0.558659,537,0.347970,3325,0.492308,325,0.492308,325,0.579607,21485,0,81,0.109014,0.414726,0.019898
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24271295,209,2301,83,1,2,146,4,1,2861,0.300049,285.200012,31,38652,18,0.610860,3094,0.468856,883,0.733906,466,0.693023,1290,0.709580,33242,1,53,0.302929,0.438708,0.020118
24271296,138,2302,22,1,5,94,1,1,2862,0.199951,267.600006,31,38635,27,0.610662,3095,0.755596,6567,0.520559,1289,0.552106,1804,0.707876,11405,0,146,0.858226,0.437576,0.020121
24271297,48084,2303,43,1,5,87,1,1,2863,0.300049,5044.600098,31,38639,22,0.610788,3096,0.747890,10309,0.520930,1290,0.552355,1805,0.708985,25742,1,94,0.849028,0.438167,0.020143
24271298,48198,2304,41,1,5,52,1,2,2864,0.300049,9833.000000,31,38643,32,0.610591,3097,0.337871,17116,0.520527,1291,0.552049,1806,0.612057,27942,0,87,0.103348,0.434652,0.020107


In [ ]:
tdf = tdf.to_numpy()
vdf = vdf.to_numpy()

In [ ]:
lgb_train = lgb.Dataset(tdf, y_train)
lgb_valid = lgb.Dataset(vdf, y_val)
del tdf, y_train
del vdf, y_val
gc.collect()

24

In [ ]:
params = {
    'objective': 'binary',
    'seed': 1103,
    'metric': ['auc'],
    'max_bin': 255,
    'learning_rate': 0.01,
    'num_leaves': 900,
    'min_data_in_leaf': 255,
#     'min_child_weight': 45.98,
#     'min_split_gain': 0.00001,
#     'lambda_l1':3.791,
#     'lambda_l2': 1.288,
#     'first_metric_only': True,
    'bagging_fraction': 0.7,
    'feature_fraction': 0.7,
    'device_type': 'gpu',
    'gpu_use_dp': True,
    'gpu_platform_id': 0,
    'gpu_device_id': 0,
}

model = lgb.train(params, 
                  lgb_train,
                  valid_sets=[lgb_valid],
                  verbose_eval=10,
                  num_boost_round=10000,
                  early_stopping_rounds=20,
                  feature_name = feat_cols,
                  categorical_feature = cat_cols
                 )

/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1555: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['num_tag', 'part', 'prev_cor', 'prev_t1', 'prior_question_had_explanation', 't1']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[LightGBM] [Info] Number of positive: 16414854, number of negative: 8585146
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 5438
[LightGBM] [Info] Number of data points in the train set: 25000000, number of used features: 29
[LightGBM] [Info] Using requested OpenCL platform 0 device 0
[LightGBM] [Info] Using GPU Device: Tesla P100-PCIE-16GB, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 16
[LightGBM] [Info] 27 dense feature groups (667.57 MB) transferred to GPU in 0.592797 secs. 1 sparse feature groups


/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.656594 -> initscore=0.648153
[LightGBM] [Info] Start training from score 0.648153
Training until validation scores don't improve for 20 rounds
[10]	valid_0's auc: 0.773554
[20]	valid_0's auc: 0.77376
[30]	valid_0's auc: 0.774258
[40]	valid_0's auc: 0.774558
[50]	valid_0's auc: 0.774739
[60]	valid_0's auc: 0.774994
[70]	valid_0's auc: 0.775173
[80]	valid_0's auc: 0.775353
[90]	valid_0's auc: 0.77551
[100]	valid_0's auc: 0.775665
[110]	valid_0's auc: 0.775903
[120]	valid_0's auc: 0.776048
[130]	valid_0's auc: 0.77617
[140]	valid_0's auc: 0.776327
[150]	valid_0's auc: 0.776489
[160]	valid_0's auc: 0.776587
[170]	valid_0's auc: 0.776757
[180]	valid_0's auc: 0.776908
[190]	valid_0's auc: 0.777038
[200]	valid_0's auc: 0.777176
[210]	valid_0's auc: 0.777313
[220]	valid_0's auc: 0.777435
[230]	valid_0's auc: 0.777573
[240]	valid_0's auc: 0.777693
[250]	valid_0's auc: 0.77783
[260]	valid_0's auc: 0.777973
[270]	valid_0's auc: 0.778089
[280]	vali

In [ ]:
print('auc:', model.best_score['valid_0']['auc'])

auc: 0.7840621291475345


In [ ]:
model.save_model("/content/drive/MyDrive/data/lgbm_model_new_mu_sigma_hung_trueskill_mean.txt")